In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from argparse import ArgumentParser, Namespace
from random import choices
import pytorch_lightning as pl
from typing import Callable, Optional, Sequence, Union
import squidpy as sq
import torch
from torch_geometric.loader import RandomNodeSampler
import pandas as pd
from torch_geometric.data import Data
from anndata import AnnData

from gpu_spatial_graph_pipeline.models.graph_embedding import GraphEmbedding
from gpu_spatial_graph_pipeline.data.node_datamodule import NodeDataModule
from gpu_spatial_graph_pipeline.data.graph_datamodule import GraphDataModule

In [8]:
def adata2data(adata:AnnData)-> Data:
    sq.gr.spatial_neighbors(adata, coord_type="generic")
    r, c = adata.obsp['spatial_connectivities'].nonzero()
    edge_index = torch.vstack([torch.from_numpy(r).to(
        torch.long), torch.from_numpy(c).to(torch.long)])
    cell_type = torch.from_numpy(pd.get_dummies(adata.obs).to_numpy())
    gene_expression = torch.from_numpy(adata.X)
    return Data(
        edge_index=edge_index,
        x=gene_expression,
        y=cell_type
    )

In [9]:
adata = sq.datasets.imc()

In [10]:
dm = NodeDataModule(adata=adata,adata2data_fn=adata2data, num_workers = 12, batch_size=100)

In [11]:
model = GraphEmbedding(num_features=34,latent_dim=30,lr=0.0001,weight_decay=0.000001)

In [12]:
trainer:pl.Trainer = pl.Trainer(accelerator='gpu',max_epochs=12,log_every_n_steps=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model,datamodule=dm)

Missing logger folder: /home/sel/Documents/projects/gpu-spatial-graph/examples/lightning_logs


TypeError: adata2data() takes 1 positional argument but 2 were given